In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)
p = np.linspace(0, 20, 100)
plt.plot(p, np.sin(15*p))

plt.show()

In [ ]:
import pandas as pd
import pandas_datareader as pdr
v = pdr.get_data_fred('GS10')


In [ ]:
v

In [ ]:
import os
import pandas_datareader.data as pdr

API_KEY = os.environ['ALPHA_VANTAGE_API_KEY']

DATE_START = '2019-01-01'
DATE_END = '2019-10-11'
SYMBOL = 'AAPL'

# We use the 'av-daily' DataReader to download data from AlphaVantage
stock = pdr.DataReader(SYMBOL, 'av-daily',
                       start=DATE_START,
                       end=DATE_END,
                       api_key=API_KEY)

print(stock)

In [ ]:
import hvplot.pandas

idx = pd.date_range('1/1/2000', periods=1000)
df  = pd.DataFrame(np.random.randn(1000, 4), index=idx, columns=list('ABCD')).cumsum()

df.hvplot()

In [ ]:
from hvplot.sample_data import us_crime

columns = ['Burglary rate', 'Larceny-theft rate', 'Robbery rate', 'Violent Crime rate']
us_crime.plot.violin(y=columns, group_label='Type of crime', value_label='Rate per 100k', invert=True)